#### Import the neccessary libraries

In [2]:
from pymongo import MongoClient
from datetime import datetime,timedelta
import pandas as pd
import random
from faker import Faker



#### Connecting to mongodb and creating Eduhub database

In [3]:
client = MongoClient('mongodb://localhost:27017/')
db = client['Eduhub_db']

### Creation of Collections schema

#### Creating user schema with validation rule

In [4]:

user_schema = {
        '$jsonSchema':{
            'bsonType':'object',
            'required': ['userId','email','firstName','lastName'],
            'properties':{
                'userId':{
                    'bsonType':'string',
                    
                    'description': 'UserId must be string and unique'
                },
                'email':{
                    'bsonType':'string',
                    
                    'description': 'Email must be string and unique'
                },
                'firstName':{
                    'bsonType':'string',
                    'description':'firstName must be  string'
                },
                'lastName':{
                    'bsonType':'string',
                    'description':'lastName must be string'
                },
                'role':{
                    'bsonType':'string',
                    'enum':['student','instructor'],
                    'description': 'Role must be student or instructor'
                    
                },
                'dateJoined':{
                    'bsonType':'date',
                    'description':'Datejoined should be date'
                    
                },
                'profile':{
                    'bsonType':'object',
                    'properties':{
                        'bio':{
                            'bsonType':'string',
                            'description':'Bio data should be string'
                        },
                        'avatar':{
                            'bsonType':'string',
                            'description':"Avatar should be string"
                        },
                        'skills':{
                            'bsonType':'string',
                            'description':'skills should string'
                        },
                        'isactive':{
                            'bsonType':'bool',
                            'description': 'Isactive must be boolean'
                        }
                    }
                }
            }
        }
    }


#### Creating User collection

In [5]:
#drop collection if exist
db.drop_collection('user')

db.create_collection('user',validator=user_schema)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Eduhub_db'), 'user')

In [6]:
db.user.create_index('userId', unique=True)

'userId_1'

In [7]:
db.user.create_index('email',unique=True)

'email_1'

#### Creating course schema

In [8]:

courses_schema = {
        '$jsonSchema': {
            'bsonType':'object',
            'required':['title'],
            'properties':{
                'courseId':{
                    'bsonType':'string',
                    
                    'description':'courseId should be a string'
                },
                'title':{
                    'bsonType':'string',
                    'description': 'title should be a string'
                },
                'description':{
                    'bsonType':'string',
                    'description':'it should be a string'
                },
                'instructorId':{            # reference to user_schema
                    'bsonType': 'string',
                    'description': 'InstructorId should be a srting' 
                },
                'category':{
                    'bsonType':'string',
                    'description': 'category should be a string'
                },
                'level':{
                    'bsonType':'string',
                    'enum': ['beginner', 'intermediate', 'advanced'],
                    'description': 'level should be beginner,intermed'
                },
                'duration':{
                    'bsonType': 'double',
                    'description':'Duration should be number in hrs'
                },
                'price':{
                    'bsonType':'double',
                    'description':'Price should be number'
                },
                'tag':{
                    'bsonType':'array',
                    'items':{
                        'bsonType':'string'
                    },
                    'description':'tag should be string'
                },
                'createdAt':{
                    'bsonType':'date',
                    'description': 'createdAT should be datetime'
                },
                'updatedAt':{
                    'bsonType':'date',
                    'description':'updatedAt should be datetime'
                },
                'isPublished':{
                    'bsonType':'bool',
                    'description': 'it should be a Boolean'
                }
            }
        }
    }






#### Creating course collection

In [9]:

db.drop_collection('courses')

db.create_collection('courses',validator=courses_schema)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Eduhub_db'), 'courses')

In [10]:
db.courses.create_index('courseId', unique=True)

'courseId_1'

#### Creating enrollment schema

In [11]:

enrollment_schema = {
        '$jsonSchema':{
            'bsonType':'object',
            'required':['userId','courseId','enrolled_on'],
            'properties':{
                'userId':{                           #references userId
                    'bsonType':'string',             
                    'description':'userId should be string that matches the userId in the user collection'
                },
                'courseId':{                        # references courses
                    'bsonType':'string',
                    'description':'courseId should be string that matches the courseId in courses collection'
                    
                },
                'enrolled_on':{
                    'bsonType':'date',
                    'description': 'enrolled_on should be date'
                }
            }
        }
    }


#### creating enrollment collection

In [12]:
# Drop enrollments collections if exits
db.drop_collection('enrollments')

db.create_collection('enrollments',validator=enrollment_schema)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Eduhub_db'), 'enrollments')

In [13]:
# To prvent duplication for same user in the same course : create a compound index

db.enrollments.create_index([('userId',1),('courseId',1)], unique=True)

'userId_1_courseId_1'

#### Creating lesson schema

In [14]:

lesson_schema = {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["courseId", "title", "content", "order"],
            "properties": {
                "courseId": {
                    "bsonType": "string",
                    "description": "courseId should be a string that references a course"
                },
                "title": {
                    "bsonType": "string",
                    "description": "title should be a string"
                },
                "content": {
                    "bsonType": "string",
                    "description": "content should be a string"
                },
                "order": {
                    "bsonType": "int",
                    "description": "order should be an integer"
                }
            }
        }
    }



#### Creating lessons collection

In [15]:
#drop lesson collection if exist
db.drop_collection('lessons')

db.create_collection('lessons', validator=lesson_schema)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Eduhub_db'), 'lessons')

In [16]:
# To make courseId and order to be unique
db.lessons.create_index([('courseId',1),('order',1)], unique=True)

'courseId_1_order_1'

#### Creating assignment schema

In [22]:

    assignment_schema = {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["lessonId", "title", "description", "due_date"],
            "properties": {
                "lessonId": {
                    "bsonType": "string",
                    "description": "lessonId should be a string and reference lessonId in lessons collection"
                },
                "title": {
                    "bsonType": "string",
                    "description": "title should be a string"
                },
                "description": {
                    "bsonType": "string",
                    "description": "description should be a string"
                },
                "due_date": {
                    "bsonType": "date",
                    "description": "due_date should be a date"
                }
            }
        }
    }



#### Create assignmnent Collection

In [23]:
# Drop assignments collection if exist
db.drop_collection('assignments')

db.create_collection('assignments', validator=assignment_schema)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Eduhub_db'), 'assignments')

#### Creating Submmission schema

In [27]:
submission_schema = {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["assignmentId", "userId", "submitted_on", "content"],
            "properties": {
                "assignmentId": {
                    "bsonType": "string",
                    "description": "assignmentId should be a string referencing an assignment"
                },
                "userId": {
                    "bsonType": "string",
                    "description": "userId should be a string referencing a user"
                },
                "submitted_on": {
                    "bsonType": "date",
                    "description": "submitted_on should be a date"
                },
                "content": {
                    "bsonType": "string",
                    "description": "content should be a string"
                }
            }
        }
    }



#### Create submission collection

In [28]:
# Drop assignments collection if exist
db.drop_collection('submissions')

db.create_collection('submissions', validator=submission_schema)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Eduhub_db'), 'submissions')

### DATA POPULATION

#### Inserting User collection samples(20) mixture of students and instructor

In [30]:
fake = Faker()

def insert_users(db, count=20):
    roles = ['student', 'instructor']
    users = []

    for i in range(count):
        role = random.choice(roles)
        user = {
            "userId": f"user_{i+1}",
            "email": fake.unique.email(),
            "firstName": fake.first_name(),
            "lastName": fake.last_name(),
            "role": role,
            "dateJoined": datetime.utcnow(),
            "profile": {
                "bio": fake.sentence(),
                "avatar": fake.image_url(),
                "skills": fake.job(),
                "isactive": random.choice([True, False])
            }
        }
        users.append(user)

    result = db.users.insert_many(users)
    print(f"{len(result.inserted_ids)} users inserted successfully.")


insert_users(db)


20 users inserted successfully.


### Inserting coursers collection sample(8) across different categories

In [31]:
#There is need for the user collection to have at lease a document with role:instructor 
#so the instructorId field can reference it. hence the below insertion:

db.users.insert_one({
    "userId": "instructor_1",
    "email": "ada@example.com",
    "firstName": "Ada",
    "lastName": "uzor",
    "role": "instructor",
    "dateJoined": datetime.utcnow(),
    "profile": {
        "bio": "Teaches coding.",
        "avatar": "http://example.com/avatar.jpg",
        "skills": "Python",
        "isactive": True
    }
})


InsertOneResult(ObjectId('684eecb9a449e5a129152022'), acknowledged=True)

In [32]:


def sample_courses(db, count=8):
    # Fetch instructor userIds
    instructor_users = list(db.users.find({"role": "instructor"}, {"userId": 1}))
    if not instructor_users:
        print("No instructors found. Please insert instructors first.")
        return

    categories = ['Technology', 'Business', 'Art', 'Science', 'Health']
    levels = ['beginner', 'intermediate', 'advanced']
    
    courses = []

    for i in range(count):
        instructor = random.choice(instructor_users)
        course = {
            "courseId": f"course_{i+1}",
            "title": fake.sentence(nb_words=4).rstrip('.'),
            "description": fake.paragraph(),
            "instructorId": instructor['userId'],
            "category": random.choice(categories),
            "level": random.choice(levels),
            "duration": round(random.uniform(1.0, 40.0), 1),
            "price": round(random.uniform(10.0, 200.0), 2),
            "tag": fake.words(nb=3),
            "createdAt": datetime.utcnow(),
            "updatedAt": datetime.utcnow(),
            "isPublished": random.choice([True, False])
        }
        courses.append(course)

    result = db.courses.insert_many(courses)
    print(f"Successfully inserted {len(result.inserted_ids)} courses.")



sample_courses(db)


Successfully inserted 8 courses.


In [ ]:
#### Inserting enrollment collections sample(15)

In [36]:


def insert_sample_enrollments(db, count=15):
    # Get student userIds
    student_users = list(db.users.find({"role": "student"}, {"userId": 1}))
    if not student_users:
        print("No students found. Please insert student users first.")
        return

    # Get courseIds
    course_list = list(db.courses.find({}, {"courseId": 1}))
    if not course_list:
        print("No courses found. Please insert courses first.")
        return

    enrollments = []
    for _ in range(count):
        user = random.choice(student_users)
        course = random.choice(course_list)

        enrollment = {
            "userId": user["userId"],
            "courseId": course["courseId"],
            "enrolled_on": datetime.utcnow()
        }

        enrollments.append(enrollment)

    result = db.enrollments.insert_many(enrollments)
    print(f"Successfully inserted {len(result.inserted_ids)} enrollments.")


insert_sample_enrollments(db)


Successfully inserted 15 enrollments.


#### Inserting lessons collection samples(25)

In [37]:


def insert_sample_lessons(db, count=25):
    # Get all available courseIds
    course_ids = list(db.courses.find({}, {"_id": 0, "courseId": 1}))
    course_ids = [c["courseId"] for c in course_ids]

    if not course_ids:
        print(" No courses found. Please insert courses first.")
        return

    lessons = []
    for i in range(count):
        lesson = {
            "courseId": random.choice(course_ids),
            "title": fake.sentence(nb_words=6),
            "content": fake.paragraph(nb_sentences=5),
            "order": random.randint(1, 20)
        }
        lessons.append(lesson)

    result = db.lessons.insert_many(lessons)
    print(f" Successfully inserted {len(result.inserted_ids)} lessons.")


insert_sample_lessons(db)


 Successfully inserted 25 lessons.


#### Inserting assignments collections sample(10)

In [38]:


def sample_assignments(db, count=10):
    # Fetch existing lessonIds
    lesson_ids = list(db.lessons.find({}, {"_id": 0, "lessonId": 1}))

    # Handle the case where lessonId might not exist yet
    if not lesson_ids:
        print(" No lessons found. Please insert lessons first.")
        return

    # If lessonId field is not set, we can assume _id or generate our own field
    # For now, we will use the "_id" field from MongoDB ObjectId
    lesson_ids = list(db.lessons.find({}, {"_id": 1}))
    lesson_ids = [str(l["_id"]) for l in lesson_ids]

    assignments = []
    for i in range(count):
        assignment = {
            "lessonId": random.choice(lesson_ids),
            "title": fake.sentence(nb_words=5),
            "description": fake.paragraph(nb_sentences=3),
            "due_date": datetime.utcnow() + timedelta(days=random.randint(1, 30))
        }
        assignments.append(assignment)

    result = db.assignments.insert_many(assignments)
    print(f"Inserted {len(result.inserted_ids)} assignments.")


sample_assignments(db)


Inserted 10 assignments.


#### Inserting submissions collection samples(12)

In [40]:


def sample_submissions(db, count=12):
    # Get assignment IDs from the 'assignments' collection
    assignments = list(db.assignments.find({}, {"_id": 1}))
    if not assignments:
        print(" No assignments found. Please insert assignments first.")
        return

    assignment_ids = [str(assignment["_id"]) for assignment in assignments]

    # Get student users from the 'user' collection
    students = list(db.user.find({"role": "student"}, {"_id": 0, "userId": 1}))
    if not students:
        print(" No students found. Please insert student users first.")
        return

    student_ids = [student["userId"] for student in students]

    # Generate submissions
    submissions = []
    for _ in range(count):
        submission = {
            "assignmentId": random.choice(assignment_ids),
            "userId": random.choice(student_ids),
            "submitted_on": datetime.utcnow() - timedelta(days=random.randint(0, 10)),
            "content": fake.paragraph(nb_sentences=3)
        }
        submissions.append(submission)

    # Insert into the 'submissions' collection
    result = db.submissions.insert_many(submissions)
    print(f"Successfully inserted {len(result.inserted_ids)} submissions.")


sample_submissions(db)


Successfully inserted 12 submissions.


#### BASIC CRUD OPERATION (# Create Operation)

In [52]:
# Add a new user

resul=db.user.insert_one({
 
 "userId": "user_1001",
    "email": "ade@example.com",
    "firstName": "Ade",
    "lastName": "Bakari",
    "role": "student",
    "dateJoined": datetime.utcnow(),
    "profile": {
        "bio": "A passionate learner interested in backend development.",
        "avatar": "https://placekitten.com/300/300",
        "skills": "Python, MongoDB",
        "isactive": True
    }
})
    
print(f'successfully inserted {resul.inserted_id} document')

successfully inserted 684f08c7a449e5a12915208d document
